<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-06 14:48:37
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.35 C
-------------------
Today PnL: -1.02 L (-0.75%)
Current PnL: -19.92 L (-13.55%)
CY Booked + Current PnL: -11.65 L (-7.92%)
-------------------
Total profit:  2.15 L
Total loss:  -22.07 L
-------------------
Total Booked + Current PnL: 15.33 L (12.77%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.13%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 78.19 L (57.94%)
Deployed:  1.20 C
Current:  1.35 C
CAGR/XIRR %: 7.64%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,761.55,-20.73,H-LC,5.28,109584.0,5272.0,12262.0,-0.32,5.05,11.19,16.81,10.0,0.43,0.83,21.91,X5K,ATH,METALS
67,SBIN,760.30,863.00,-11.51,M-LC,3.96,213166.0,11686.0,15518.0,0.47,5.80,7.28,13.51,62.0,0.75,1.62,19.13,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,21.15,H-SC,13.50,155301.0,29101.0,17068.0,0.90,23.06,10.99,36.59,156.0,1.71,1.18,38.46,XY24,NTT,HEALTHCARE
82,TTKPRESTIG,769.29,770.00,-8.91,H-SC,11.64,83231.0,-17546.0,17637.0,1.07,-17.41,21.19,0.09,145.0,-0.99,0.63,9.35,OX40N,NTT,DURABLES
71,SIS,477.00,477.00,2256.48,H-SC,7.04,63393.0,-17697.0,17699.0,-0.33,-21.82,27.92,0.00,157.0,-1.00,0.48,28.48,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,561.33,680.00,-19.06,H-MC,9.91,229088.0,1749.0,46322.0,-1.06,0.77,20.22,21.14,107.0,0.04,1.74,29.13,XY24,NTT,PAINTS
58,PGHH,13388.00,18062.58,-21.14,H-MC,6.85,199050.0,-1770.0,71897.0,-0.64,-0.88,36.12,34.92,84.0,-0.02,1.51,3.75,X40,ATH,FMCG
56,NATIONALUM,189.63,247.44,-50.15,H-MC,2.87,99910.0,-594.0,31232.0,0.67,-0.59,31.26,30.49,81.0,-0.02,0.76,31.62,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.50,H-LC,1.81,203760.0,-740.0,42158.0,-0.83,-0.36,20.69,20.25,9.0,-0.02,1.55,8.34,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-14.67,H-LC,2.63,169388.0,-31998.0,119605.0,2.22,-15.89,70.61,43.51,19.0,-0.27,1.29,17.91,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-5.66,H-LC,3.25,181386.0,-18924.0,78903.0,-1.81,-9.45,43.50,29.94,24.0,-0.24,1.38,25.31,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.42,H-LC,3.75,185910.0,-14596.0,63377.0,-0.76,-7.28,34.09,24.33,30.0,-0.23,1.41,7.21,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-17.54,H-LC,4.18,261390.0,-4036.0,60956.0,-1.46,-1.52,23.32,21.44,26.0,-0.07,1.99,5.02,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UNIONBANK,123.87,163.00,-15.25,M-LC,15.61,148936.0,8096.0,36400.0,2.35,5.75,24.44,31.59,72.0,0.22,1.13,33.76,XY24,NTT,BANKS
5,ASIANPAINT,2961.56,4250.00,-14.67,H-LC,2.63,169388.0,-31998.0,119605.0,2.22,-15.89,70.61,43.51,19.0,-0.27,1.29,17.91,X40,BTT,PAINTS
2,ACC,2579.05,3906.00,-38.07,M-MC,4.88,133663.0,-54608.0,151480.0,1.62,-29.00,113.33,51.45,178.0,-0.36,1.02,2.38,XY24,BTT,CEMENT
54,MASFIN,326.60,399.50,-19.15,H-SC,16.35,91920.0,-6060.0,27934.0,1.44,-6.18,30.39,22.32,151.0,-0.22,0.70,33.32,XR,ATH,FINANCE
42,INDIAMART,2327.09,4850.92,-48.82,H-SC,13.38,133242.0,9906.0,123862.0,1.09,8.03,92.96,108.45,118.0,0.08,1.01,32.18,AR,ATH,MISC


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BRITANNIA,5502.66,6362.67,7.97,H-LC,9.28,193860.0,-4236.0,35205.0,-4.45,-2.14,18.16,15.63,31.0,-0.12,1.48,19.24,XY25,ATH,FMCG
60,RAJESHEXPO,517.65,518.00,1665.15,L-SC,3.42,48177.0,-89000.0,89094.0,-3.87,-64.88,184.93,0.07,266.0,-1.00,0.37,19.82,OX40N,NTT,JEWELLERY
75,SYMPHONY,1306.42,1306.00,-39.43,M-SC,9.43,129140.0,-42001.0,41945.0,-3.37,-24.54,32.48,-0.03,197.0,-1.00,0.98,0.00,OX40N,NTT,DURABLES
74,SURYODAY,189.78,240.00,40.35,H-SC,5.06,99350.0,-50576.0,90250.0,-3.07,-33.73,90.84,26.46,142.0,-0.56,0.76,27.27,XR,NTT,BANKS
76,TANLA,917.30,1963.11,-30.09,H-SC,7.63,170694.0,-68721.0,341678.0,-2.69,-28.70,200.17,114.01,133.0,-0.20,1.30,56.55,AR,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,129.91,H-SC,4.66,150796.0,-23763.0,23796.0,0.92,-13.61,15.78,0.02,135.0,-1.00,1.15,30.76,OX40N,NTT,PAINTS
82,TTKPRESTIG,769.29,770.0,-8.91,H-SC,11.64,83231.0,-17546.0,17637.0,1.07,-17.41,21.19,0.09,145.0,-0.99,0.63,9.35,OX40N,NTT,DURABLES
83,UJJIVANSFB,52.77,53.0,34.19,H-SC,20.64,116127.0,-26352.0,26976.0,-2.23,-18.50,23.23,0.44,163.0,-0.98,0.88,37.06,OX40N,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-22.02,H-MC,2.05,81809.0,-20843.0,20837.0,-1.98,-20.30,25.47,-0.00,101.0,-1.00,0.62,24.36,OX40N,NTT,IT
49,KANSAINER,299.63,340.0,-69.48,H-SC,1.66,212220.0,-57447.0,93780.0,-2.63,-21.30,44.19,13.47,136.0,-0.61,1.61,7.34,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.38,H-LC,14.87,236886.0,-59048.0,152957.0,-0.88,-19.95,64.57,31.73,1.0,-0.39,1.80,1.13,X40,BTT,IT
88,VBL,492.64,671.64,-11.37,H-LC,11.34,269027.0,3987.0,92303.0,-1.58,1.50,34.31,36.33,3.0,0.04,2.05,15.00,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-19.96,H-LC,8.68,257223.0,-4378.0,150012.0,-1.60,-1.67,58.32,55.67,4.0,-0.03,1.96,4.33,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.44,H-LC,8.74,198432.0,-18087.0,111003.0,0.08,-8.35,55.94,42.91,5.0,-0.16,1.51,3.72,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,0.53,H-LC,9.08,178693.0,-11953.0,58468.0,-1.21,-6.27,32.72,24.40,8.0,-0.20,1.36,8.67,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,21.15,H-SC,13.5,155301.0,29101.0,17068.0,0.9,23.06,10.99,36.59,156.0,1.71,1.18,38.46,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,30.30,H-SC,13.54,133283.0,7986.0,51087.0,-0.68,6.37,38.33,47.15,144.0,0.16,1.01,66.75,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,21.15,H-SC,13.50,155301.0,29101.0,17068.0,0.90,23.06,10.99,36.59,156.0,1.71,1.18,38.46,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-2.86,H-MC,4.90,156432.0,5519.0,51779.0,-1.70,3.66,33.10,37.97,90.0,0.11,1.19,29.89,AR,ATH,AUTO
19,CAMS,3643.00,5226.82,-4.80,H-SC,1.77,107016.0,5012.0,39339.0,0.67,4.91,36.76,43.48,120.0,0.13,0.81,22.52,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2326.38,-6.96,H-SC,8.68,174600.0,9930.0,58037.0,-0.27,6.03,33.24,41.28,119.0,0.17,1.33,14.49,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,30.30,H-SC,13.54,133283.0,7986.0,51087.0,-0.68,6.37,38.33,47.15,144.0,0.16,1.01,66.75,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,-2.86,H-MC,4.90,156432.0,5519.0,51779.0,-1.70,3.66,33.10,37.97,90.0,0.11,1.19,29.89,AR,ATH,AUTO
90,WHIRLPOOL,1219.98,2270.00,-40.95,M-SC,5.70,99375.0,7876.0,70874.0,0.59,8.61,71.32,86.07,211.0,0.11,0.76,43.76,XR,NTT,DURABLES
42,INDIAMART,2327.09,4850.92,-48.82,H-SC,13.38,133242.0,9906.0,123862.0,1.09,8.03,92.96,108.45,118.0,0.08,1.01,32.18,AR,ATH,MISC
56,NATIONALUM,189.63,247.44,-50.15,H-MC,2.87,99910.0,-594.0,31232.0,0.67,-0.59,31.26,30.49,81.0,-0.02,0.76,31.62,MH,ATH,METALS


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.00,2256.48,H-SC,7.04,63393.0,-17697.0,17699.0,-0.33,-21.82,27.92,0.00,157.0,-1.00,0.48,28.48,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.00,-55.73,L-SC,21.59,66240.0,-47309.0,87357.0,-2.07,-41.66,131.88,35.27,267.0,-0.54,0.50,62.12,XR,NTT,HOTELS
54,MASFIN,326.60,399.50,-19.15,H-SC,16.35,91920.0,-6060.0,27934.0,1.44,-6.18,30.39,22.32,151.0,-0.22,0.70,33.32,XR,ATH,FINANCE
3,ALKYLAMINE,2347.98,4546.37,-0.30,H-SC,18.79,92846.0,-8117.0,102651.0,-2.47,-8.04,110.56,93.63,149.0,-0.08,0.71,38.29,SR,ATH,CHEMICALS
36,HINDALCO,651.95,761.55,-20.73,H-LC,5.28,109584.0,5272.0,12262.0,-0.32,5.05,11.19,16.81,10.0,0.43,0.83,21.91,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-23.40,H-SC,7.01,217967.0,24947.0,23083.0,0.53,12.92,10.59,24.89,128.0,1.08,1.66,39.29,X40N,NTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,21.15,H-SC,13.50,155301.0,29101.0,17068.0,0.90,23.06,10.99,36.59,156.0,1.71,1.18,38.46,XY24,NTT,HEALTHCARE
69,SHALBY,261.39,327.00,939.87,M-SC,8.41,141455.0,-39688.0,85156.0,-1.31,-21.91,60.20,25.10,233.0,-0.47,1.08,13.36,XY24,NTT,HEALTHCARE
88,VBL,492.64,671.64,-11.37,H-LC,11.34,269027.0,3987.0,92303.0,-1.58,1.50,34.31,36.33,3.0,0.04,2.05,15.00,X40N,ATH,FMCG
27,DIXON,14367.00,18940.15,-28.08,H-MC,6.62,132416.0,17480.0,19108.0,-1.76,15.21,14.43,31.83,85.0,0.91,1.01,49.02,X40N,ATH,IT


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.67
1,25,41.46
2,50,71.41


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.68
LC    37.12
MC    22.19
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.97
X40N     15.17
X40      13.29
XY25     11.70
XR        8.98
AR        7.68
OX40N     7.68
X5K       2.22
MH        1.77
X200      1.38
SR        1.15
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.98
H-SC    27.36
H-MC    19.20
M-SC    11.90
M-LC     7.08
M-MC     2.64
L-SC     1.42
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.22,-3.13,36.29
IT,11.98,-18.43,75.95
FINANCE,8.35,-18.86,61.99
BANKS,7.50,-15.63,62.80
PAINTS,5.79,-14.64,37.23
HEALTHCARE,5.12,2.46,22.17
MISC,5.11,-21.34,92.33
ELECTRICAL,4.70,-13.72,62.97
AUTO,4.39,-17.22,68.21


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2831569.0,23
XR,1114034.0,13
AR,1055520.0,9
X40,825570.0,10
X40N,612020.0,12
XY25,512050.0,8
OX40N,367074.0,10
SR,192709.0,2
X5K,146999.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2725302.0,30
M-SC,1405938.0,15
H-LC,1346350.0,20
H-MC,1183058.0,15
M-LC,447547.0,5
M-MC,331660.0,2
L-SC,274765.0,3
L-MC,56815.0,1
L-LC,47332.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      948414.0      7
M-SC       XY24      859303.0      7
H-SC       AR        611892.0      4
H-LC       X40       548583.0      6
H-SC       XR        519847.0      6
H-MC       XY24      463706.0      4
M-MC       XY24      331660.0      2
H-LC       X40N      313000.0      6
           AR        233175.0      2
H-SC       X40N      215304.0      4
H-MC       XY25      210081.0      2
           X40       201573.0      3
H-SC       SR        192709.0      2
           OX40N     186049.0      6
L-SC       XR        185671.0      2
M-LC       XY24      185656.0      2
M-SC       AR        158674.0      2
M-LC       X5K       134737.0      1
M-SC       XY25      121522.0      1
H-MC       XR        120134.0      1
M-SC       XR        119931.0      2
H-LC       XY25      117597.0      3
M-LC       XR        111636.0      1
L-SC       OX40N      89094.0      1
H-MC       X40N       83716.0      2
H-LC       X200       78903.0      1
M-SC       X40        75414.0      1
           OX40N      71094.0      2
L-MC       XR         56815.0      1
H-MC       AR         51779.0      1
H-SC       MH         51087.0      1
L-LC       XY25       47332.0      1
H-LC       XY24       42830.0      1
H-MC       MH         31232.0      1
           OX40N      20837.0      1
M-LC       XY25       15518.0      1
H-LC       X5K        12262.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
